In [1]:
!pip3 install inflect
import json
import nltk
import os
import random
import re
import torch
import gensim
import string
import inflect
import torch.nn.functional as F
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from torch import nn, optim
from bs4 import BeautifulSoup
from IPython.display import display
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
p = inflect.engine()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
english_words = set(words.words())
def remove_non_english_words(text):
    words_list = text.split()
    filtered_words = [word for word in words_list if word.lower() in english_words]
    return ' '.join(filtered_words)

In [37]:
def cleanReview(text):
  txt = text.translate(str.maketrans('','',string.punctuation))
  txt = remove_non_english_words(txt)
  return txt

def readFiles(filename):
    with open(filename,'r') as f:
        texto = f.read()  
    # Convert all text to lowercase
    texto = texto.lower()
    # Replace newline characters with empty string
    texto = texto.replace('\n', '')
    # Find all tags in the text and remove closing tags
    textohelp = re.sub(r'</.*?>', '', texto)
    # Extract names of all tags in the text
    etiquetas = re.findall(r'<.*?>', textohelp)
    nombres_etiquetas = [re.sub(r'[<>]', '', etiqueta) for etiqueta in etiquetas]
    # Find unique tags
    etiquetas_unicas = set(nombres_etiquetas)
    # Create a BeautifulSoup object from the text
    soup = BeautifulSoup(texto, 'html.parser')
    # Find all review elements and create a Python object
    reviews =soup.find_all('review')
    return  reviews

def loadReviews(revs,reviewType):
  reviewList = []
  for a in revs:
        objeto = {}
        # Replace newline characters with empty string
        textoaux = a.prettify().replace('\n', '')
        if len(textoaux)>3000 or len(textoaux)<10 :
            continue
        try:
             # code that might raise an error if read a unknown character
            raiz = ET.fromstring(textoaux)
        except:
            continue
        # Mapping the new object
        if raiz.find('rating') is not None:
            objeto['rating'] = raiz.find('rating').text
        if raiz.find('review_text') is not None:
            objeto['review_text'] = cleanReview(raiz.find('review_text').text)
        objeto['result']=reviewType
        
        reviewList.append(objeto)
  return reviewList


# result = 0 NEGATIVE
raw_dataseta = readFiles('./negative.review')
negRevs = loadReviews(raw_dataseta,0)

# result = 1 POSITIVE
raw_datasetb =  readFiles('./positive.review')
posRevs = loadReviews(raw_datasetb,1)

# Full dataset NEG & POS reviews
data = negRevs + posRevs 
df = pd.DataFrame(data)
dataset = df.dropna()
dataset = dataset.copy()

print(len(dataset))
# Randomize the dataset
dataset = dataset.reindex(np.random.permutation(dataset.index))
dataset.head()

1954


,rating,review_text,result
487,1.0,these were are a p o you dont know what is it ...,0
130,1.0,though i like the keyboard found two serious w...,0
684,1.0,i bought this product less than a year ago day...,0
378,2.0,it flashy and on the web shopping that feature...,0
1099,5.0,great quality excellent construction and stron...,1


In [38]:
# Remove repeating words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))

In [39]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [40]:
temp = []
#Splitting pd.Series to list
data_to_list = dataset['review_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append((data_to_list[i]))
data_words = list(sent_to_words(temp))
print(data_words[:5])
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

[['these', 'were', 'are', 'you', 'dont', 'know', 'what', 'is', 'it', 'because', 'cant', 'say', 'it', 'on', 'basically', 'the', 'layer', 'an', 'offset', 'the', 'do', 'not', 'match', 'in', 'position', 'this', 'will', 'give', 'you', 'if', 'you', 'have', 'file', 'which', 'is', 'directly', 'on', 'layer', 'break', 'or', 'at', 'the', 'end', 'of', 'the', 'disk', 'do', 'not', 'buy', 'repeat', 'do', 'not', 'not', 'sure', 'why', 'their', 'crummy', 'product'], ['though', 'like', 'the', 'keyboard', 'found', 'two', 'serious', 'with', 'the', 'mouse', 'first', 'it', 'is', 'very', 'sensitive', 'to', 'movement', 'tried', 'making', 'some', 'but', 'it', 'is', 'still', 'difficult', 'to', 'control', 'finally', 'and', 'more', 'importantly', 'the', 'mouse', 'set', 'of', 'aa', 'about', 'every', 'days', 'only', 'use', 'the', 'mouse', 'about', 'week', 'so', 'should', 'be', 'getting', 'lot', 'more', 'battery', 'life', 'bought', 'other', 'and', 'have', 'been', 'with', 'them', 'but', 'in', 'this', 'case', 'wish', '

In [41]:
# function to clean the data and deleate stop words
def text_procesing(text):
  STOPWORDS = set(stopwords.words('english'))
  cleaned_reviews = []
  for review in text:
    tokens = [word for word in word_tokenize(review) if not word in STOPWORDS]
    cleaned_reviews.append(" ".join(tokens))
  lemmatizer = WordNetLemmatizer()
  lem_reviews = []
  for review in cleaned_reviews:
    lem_reviews.append(" ".join(list(map(lemmatizer.lemmatize, word_tokenize(review)))))
  return lem_reviews

In [42]:
y = dataset['review_text']
dataset['review_text'] =text_procesing(y)

In [43]:
max_words = 5000
max_len = 200
# Create the token array and their sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dataset['review_text'])
sequences = tokenizer.texts_to_sequences(dataset['review_text'])
reviews_sequences = sequence.pad_sequences(sequences, maxlen=max_len)
print(reviews_sequences)

[[   0    0    0 ...  116 3318   10]
 [   0    0    0 ...   12   43  121]
 [   0    0    0 ...  114  123  106]
 ...
 [   0    0    0 ... 2090  583  550]
 [   0    0    0 ...   95  119    7]
 [   0    0    0 ... 1290  693  436]]


In [44]:
# Y input variable sentiment categories 
result_sentiment = keras.utils.to_categorical(dataset['result'].astype('float32'), 2, dtype="float32")
print(result_sentiment)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [45]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(reviews_sequences,result_sentiment, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

1465 489 1465 489


In [46]:
# Creation of the NN Sequential model
model0 = Sequential()
embedding_layer = Embedding(1000, 64)
model0.add(layers.Embedding(max_words, 15))
model0.add(layers.SimpleRNN(15,return_sequences=True))
model0.add(layers.SimpleRNN(15))
model0.add(layers.Dense(2,activation='softmax'))

In [47]:
md = Sequential()
embedding_layer = Embedding(1000, 64)
md.add(layers.Embedding(max_words, 15))
md.add(layers.SimpleRNN(15,return_sequences=True))
md.add(layers.SimpleRNN(15))
md.add(layers.Dense(2,activation='softmax'))

In [48]:
md.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
# history = model0.fit(X_train, y_train, epochs=10,callbacks=[PrintDot()])
history = md.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

Epoch 1/10
46/46 [==============================] - 8s 133ms/step - loss: 0.6995 - accuracy: 0.5358 - val_loss: 0.6807 - val_accuracy: 0.5440
Epoch 2/10
46/46 [==============================] - 4s 96ms/step - loss: 0.5862 - accuracy: 0.7215 - val_loss: 0.7035 - val_accuracy: 0.5603
Epoch 3/10
46/46 [==============================] - 5s 108ms/step - loss: 0.3824 - accuracy: 0.8628 - val_loss: 0.7745 - val_accuracy: 0.5665
Epoch 4/10
46/46 [==============================] - 5s 112ms/step - loss: 0.2049 - accuracy: 0.9352 - val_loss: 0.8522 - val_accuracy: 0.5849
Epoch 5/10
46/46 [==============================] - 4s 89ms/step - loss: 0.1035 - accuracy: 0.9713 - val_loss: 0.9842 - val_accuracy: 0.5542
Epoch 6/10
46/46 [==============================] - 6s 131ms/step - loss: 0.0565 - accuracy: 0.9870 - val_loss: 1.1143 - val_accuracy: 0.5419
Epoch 7/10
46/46 [==============================] - 4s 96ms/step - loss: 0.0327 - accuracy: 0.9904 - val_loss: 1.2088 - val_accuracy: 0.5501
Epoch 8/1

In [49]:
model0.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
# history = model0.fit(X_train, y_train, epochs=10,callbacks=[PrintDot()])
history = model0.fit(X_train, y_train, epochs=10)
# validation_data=(X_test, y_test))
# history = model0.fit(X_train, y_train, epochs=10,validation_split=0.2)

Epoch 1/10
46/46 [==============================] - 7s 95ms/step - loss: 0.6872 - accuracy: 0.5604
Epoch 2/10
46/46 [==============================] - 4s 80ms/step - loss: 0.5093 - accuracy: 0.7788
Epoch 3/10
46/46 [==============================] - 4s 96ms/step - loss: 0.2917 - accuracy: 0.8962
Epoch 4/10
46/46 [==============================] - 5s 100ms/step - loss: 0.1657 - accuracy: 0.9522
Epoch 5/10
46/46 [==============================] - 4s 83ms/step - loss: 0.0842 - accuracy: 0.9768
Epoch 6/10
46/46 [==============================] - 4s 91ms/step - loss: 0.0461 - accuracy: 0.9870
Epoch 7/10
46/46 [==============================] - 5s 102ms/step - loss: 0.0278 - accuracy: 0.9925
Epoch 8/10
46/46 [==============================] - 4s 83ms/step - loss: 0.0136 - accuracy: 0.9973
Epoch 9/10
46/46 [==============================] - 4s 91ms/step - loss: 0.0098 - accuracy: 0.9986
Epoch 10/10
46/46 [==============================] - 5s 107ms/step - loss: 0.0076 - accuracy: 0.9986


PREDICTIONS

In [50]:
sentiment = ['Negative','Positive']

In [74]:
review_input = input("WRITE THE REVIEW:")

WRITE THE REVIEW:I never even traveled with it. It just stopped working completely making me lose data.  NEVER AGAIN I'll buy anything from this company. This drive is a piece of rubbish!


In [51]:
# NEGATIVE Test reviewa
review_input= 'the product is horrible, and useless'
# review_input= 'this experience has been the worst , want all my money back'
# review_input='I never even traveled with it. It just stopped working completely making me lose data. NEVER AGAIN I'll buy anything from this company. This drive is a piece of rubbish!'

# POSITIVE Test reviewa
# review_input = 'The product makes good smothies and also is great for juices'
# review_input= 'definitely recommend this memory card, download time is great'
# review_input= 'I borrowed this book on CD from the library and half way through, ordered the book so I would have it to refer to. Excellent book, highly recommended. Helps to put a realistic perspective on millionaires'



In [75]:
sequencer= tokenizer.texts_to_sequences([review_input])

In [76]:
# MODEL WITHOUT VALIDATION DATA
test= sequence.pad_sequences(sequencer, maxlen=max_len)
predictions =model0.predict(test) 
print(predictions)
sentiment[np.around(predictions, decimals=0).argmax(axis=1)[0]]

1/1 [==============================] - 0s 63ms/step
[[9.992968e-01 7.031906e-04]]


'Negative'

In [77]:
# MODEL WITH VALIDATION DATA
test= sequence.pad_sequences(sequencer, maxlen=max_len)
predictions =md.predict(test) 
print(predictions)
sentiment[np.around(predictions, decimals=0).argmax(axis=1)[0]]

1/1 [==============================] - 0s 35ms/step
[[0.99721354 0.0027865 ]]


'Negative'